Using SQLAlchemy to model table schemas and create a sqlite database for the two tables.

In [1]:
import pandas as pd
from datetime import date, datetime
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, FLOAT,DATETIME, DateTime
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [2]:
df_hm = pd.read_csv("Cleaned_CSV/hawaii_measurements_c.csv")
df_hs = pd.read_csv("Cleaned_CSV/hawaii_stations.csv")
df_hm.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-07,0.06,70


In [3]:
df_hs.head()

,Unnamed: 0,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [14]:
# converting date to datetime object
df_hm['date'] = df_hm['date'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d"))
df_hm.dtypes

TypeError: strptime() argument 1 must be str, not Timestamp

In [5]:
class Measurement(Base):
    __tablename__ = 'measurement'
    __table_args__ = {'extend_existing': True}  # to replace the existing table
    measurement_id = Column(Integer, autoincrement = True, primary_key=True)
    station_id = Column(String)
    date = Column(DateTime)
    prcp = Column(FLOAT)
    tobs = Column(Integer) 
    

In [6]:
class Station(Base):
    __tablename__= 'station'
    __table_args__ = {'extend_existing': True} 
    station_id = Column(String, primary_key = True)
    name = Column(String)
    latitude = Column(FLOAT)
    longitude = Column(FLOAT)
    elevation = Column(FLOAT)

In [7]:
engine = create_engine("sqlite:///Resources_cleaned/hawaii.sqlite", echo=False)


In [8]:
# create tables in database
Base.metadata.create_all(engine)

In [9]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

In [10]:
# adding value que in class Measurement
for index, row in df_hm.iterrows():
    session.add(Measurement(station_id=row['station'], date=row['date'], prcp=row['prcp'], tobs=row['tobs']))

In [11]:
# adding value que in class Station
for index, row in df_hs.iterrows():
    session.add(Station(station_id=row['station'], name=row['name'], latitude=row['latitude'], longitude=row['longitude'],
                       elevation=row['elevation']))

In [12]:
# commiting the additions to the db (flushing out the data qued to be added)
session.commit()